# FEniCS simulation of Eshelby's circular inclusion problem

The aim of this notebook is to setup a very simple FEniCS simulation. The framework is linear, plane strain elasticity. We model a matrix in a disk around the origin (radius $R_m$) with an inclusion having the shape of another disk around the origin, with a smaller radius ($R_i < R_e$). The matrix and the inclusion have different elastic modulus ($E$: Young modulus; $\nu$: Poisson ratio) but are both isotropic and linearly elastic.

\begin{equation}
\sigma_{ij} = \lambda\varepsilon_{kk}\delta_{ij}+2\mu\varepsilon_{ij},
\end{equation}

where indices $i, j, k$ are restricted to $\{1, 2\}$ and $\lambda$, $\mu$ are the Lamé coefficients :

\begin{equation*}
\mu=\frac{E}{2\bigl(1+\nu\bigr)}
\quad\text{and}\quad
\lambda=\frac{2\mu\nu}{1-2\nu}.
\end{equation*}

The variational formulation of the problem is the following:

Find $u\in \mathcal{C}\equiv\{u: H^1(\Omega), \; u(x_1,x_2)|_{x_1^2+x_2^2=R_e^2}
%\text{border}
=(-x_2,-x_1)\}$ such that 
$\forall v\in \mathcal{C}_0\equiv \mathcal{C}$


\begin{equation}
\int_\Omega \sigma(\varepsilon(u)):\varepsilon(v)\,\mathrm{d}x\,\mathrm{d}y =
\int_{\Omega} b \cdot v\,\mathrm{d} x\,\mathrm{d} y,
\end{equation}

where the body force $b=0$ and $\sigma(\varepsilon)$ is the constitutive equation and $\varepsilon(u)=\mathrm{sym} (\nabla u)$  

![shema](inclusion_shear.png)

you can find help here:

- Mesh generation
   - https://jorgensd.github.io/dolfinx-tutorial/chapter1/membrane_code.html
   - https://jorgensd.github.io/dolfinx-tutorial/chapter2/ns_code2.html#mesh-generation
   - https://docs.fenicsproject.org/dolfinx/main/python/demos/demo_gmsh.html
- Dirichlet bc
    - https://jorgensd.github.io/dolfinx-tutorial/chapter1/fundamentals_code.html#defining-the-boundary-conditions
    - https://jorgensd.github.io/dolfinx-tutorial/chapter2/linearelasticity_code.html#boundary-conditions
    - https://jorgensd.github.io/dolfinx-tutorial/chapter3/component_bc.html
- Visualization in Paraview
    - https://jorgensd.github.io/dolfinx-tutorial/chapter1/membrane_paraview.html
- Interpolating the strain tensor once we have the displacement vector
    - https://jorgensd.github.io/dolfinx-tutorial/chapter2/linearelasticity_code.html#stress-computation
- pyvista
    - https://jorgensd.github.io/dolfinx-tutorial/chapter2/linearelasticity_code.html#visualization
    - https://docs.fenicsproject.org/dolfinx/main/python/demos/demo_pyvista.html
- The Assemble function to perform integrals
    - https://jorgensd.github.io/dolfinx-tutorial/chapter1/fundamentals_code.html#computing-the-error
- Convergence study
    - https://jorgensd.github.io/dolfinx-tutorial/chapter4/convergence.html

In [ ]:
from mpi4py import MPI
import dolfinx # FEM in python
import matplotlib.pyplot as plt
import ufl # variational formulations
import numpy as np
import gmsh
import basix.ufl
import dolfinx.fem as fem
import dolfinx.fem.petsc

In [ ]:
import pyvista # visualisation in python notebook
pyvista.start_xvfb()
pyvista.set_jupyter_backend("static")

In [ ]:
import utils # this to be able to have the value of a solution at point (x,y)
# use it this way : utils.evaluate_at_points(np.array([[x], [y], [z]]), u_solution)

# Parameters of the simulation

In [ ]:
# Geometry
R_i = 1.0 # Radius of the inclusion
R_e = 6.9  # Radius of the matrix (whole domain)
aspect_ratio = 1.0 # start with a circle, otherwise ellipse

In [ ]:
# Material
E_m = 1.0 # Young's modulus in matrix
nu_m = 0.35 # Poisson's ratio in matrix
E_i = 11.0 # Young's modulus of inclusion
nu_i = 0.3 # Poisson's ratio in inclusion

## Create the mesh with gmsh

In [ ]:
R_i = 1.0 # Radius of the inclusion
R_e = 6.9  # Radius of the matrix (whole domain)
aspect_ratio = 1.0
mesh_size = 0.2*R_i
mesh_order = 1 

mesh_comm = MPI.COMM_WORLD
model_rank = 0
gmsh.initialize()
facet_names = {"inner_boundary": 1, "outer_boundary": 2}
cell_names = {"inclusion": 1, "matrix": 2}
model = gmsh.model()
model.add("Disk")
model.setCurrent("Disk")
gdim = 2 # geometric dimension of the mesh
inner_disk = gmsh.model.occ.addDisk(0, 0, 0, R_i, aspect_ratio * R_i)
outer_disk = gmsh.model.occ.addDisk(0, 0, 0, R_e, R_e)
whole_domain = gmsh.model.occ.fragment(
            [(gdim, outer_disk)], [(gdim, inner_disk)]
        )
gmsh.model.occ.synchronize()
# Add physical tag for bulk
inner_domain = whole_domain[0][0]
outer_domain = whole_domain[0][1]
model.addPhysicalGroup(gdim, [inner_domain[1]], tag=cell_names["inclusion"])
model.setPhysicalName(gdim, inner_domain[1], "Inclusion")
model.addPhysicalGroup(gdim, [outer_domain[1]], tag=cell_names["matrix"])
model.setPhysicalName(gdim, outer_domain[1], "Matrix")

# Add physical tag for boundaries
lines = gmsh.model.getEntities(dim=1)
inner_boundary = lines[1][1]
outer_boundary = lines[0][1]
gmsh.model.addPhysicalGroup(1, [inner_boundary], facet_names["inner_boundary"])
gmsh.model.addPhysicalGroup(1, [outer_boundary], facet_names["outer_boundary"])
gmsh.option.setNumber("Mesh.CharacteristicLengthMin",mesh_size)
gmsh.option.setNumber("Mesh.CharacteristicLengthMax",mesh_size)
model.mesh.generate(gdim)
gmsh.option.setNumber("General.Terminal", 1)
model.mesh.setOrder(mesh_order)
gmsh.option.setNumber("General.Terminal", 0)

# Import the mesh in dolfinx
from dolfinx.io import gmshio
domain, cell_tags, facet_tags = gmshio.model_to_mesh(model, mesh_comm, model_rank, gdim=gdim)
domain.name = "composite"
cell_tags.name = f"{domain.name}_cells"
facet_tags.name = f"{domain.name}_facets"
gmsh.finalize()

In [17]:
cell_names["matrix"]

2

In [16]:
cell_names["inclusion"]

1

# `Questions start here`

# 0) Export the mesh as a xdmf file and open it in Paraview

In [ ]:
# Save the mesh in XDMF format
with dolfinx.io.XDMFFile(MPI.COMM_WORLD, "output/mesh.xdmf", "w") as file:
    file.write_mesh(domain)
    domain.topology.create_connectivity(1, 2)

# 1) Plot the mesh with a color code to locate the inclusion and the matrix

In [ ]:
topology, cells, geometry = dolfinx.plot.vtk_mesh(domain)
function_grid = pyvista.UnstructuredGrid(topology, cells, geometry)
function_grid["Marker"] = cell_tags.values
plotter = pyvista.Plotter()
plotter.add_mesh(function_grid, show_edges=True)
plotter.show_bounds(grid='front', location='outer', all_edges=True)
plotter.view_xy()
plotter.show()

# 2) Define integral over the two domains domains

In [ ]:
ds = ufl.Measure("ds", XXXX)
dx = ufl.Measure("dx", XXXX)
one = dolfinx.fem.Constant(domain,dolfinx.default_scalar_type(1.))

In [ ]:
one = dolfinx.fem.Constant(domain,dolfinx.default_scalar_type(1.))
area_domain = utils.assemble_scalar(dolfinx.fem.form(one * dx))
area_inclusion = utils.assemble_scalar(dolfinx.fem.form(one * dx(XXX)))
area_matrix = utils.assemble_scalar(dolfinx.fem.form(one * dx(XXX)))
area_inclusion, area_matrix, area_domain

# 3) Define the elastic problem

In [ ]:
V = fem.functionspace(domain, ("Lagrange", XXX, (domain.XXX, )))

def eps(u):
    return XXX.sym(ufl.grad(u))

I2 = ufl.Identity(2)

# Hooke's law is written as the top of this notebook
def sigma(eps, E, nu):
    mu = XXXX
    lamb = XXXX
    return lamb*ufl.tr(eps)*I2 + 2*mu*eps

u = ufl.TrialFunction(V)
u_bar = ufl.TestFunction(V)

bilinear_form_inclusion = XXXX
bilinear_form_matrix = XXXX
bilinear_form = bilinear_form_inclusion + bilinear_form_matrix
g=0.0 # no weight
body_force = dolfinx.fem.Constant(domain, dolfinx.default_scalar_type((0,-g)))
linear_form = ( ufl.dot(body_force,u_bar)  ) * ufl.dx

# 4) Boundary condition

In [ ]:
# this finds the label of the degree of freedom for the nodes on the boundary facets
outer_facets = facet_tags.find(XXXX)
print("tags:", outer_facets)
outer_boundary_dofs = dolfinx.fem.locate_dofs_topological(XXX)
print("dofs:",outer_boundary_dofs)

In [ ]:
uD = dolfinx.fem.Function(V)
u_on_boundary = lambda x: np.array([-x[1], -x[0]], dtype=uD.x.array.dtype)
uD.interpolate(u_on_boundary)
bc = dolfinx.fem.dirichletbc(uD, XXX) # pas de ,V

In [ ]:
problem = dolfinx.fem.petsc.LinearProblem(XXX, XXX, bcs=[bc], 
                                          petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
u_solution = problem.solve()

# 4.5) Plot the solution using pyvista

In [ ]:
# Create pyvista grid
topology, cells, geometry = dolfinx.plot.vtk_mesh(u_solution.function_space)
function_grid = pyvista.UnstructuredGrid(topology, cells, geometry)

plotter = pyvista.Plotter() # create plotter

# we add the mesh to the plot #grid is defined above
plotter.add_mesh(function_grid, show_edges=True, style="wireframe", color="k")

# add the deformed shape
values = np.zeros((geometry.shape[0], 3))
values[:, :len(u_solution)] = u_solution.x.array.reshape(geometry.shape[0], len(u_solution))
function_grid["u"] = values
function_grid.set_active_vectors("u")
warped = function_grid.warp_by_vector("u", factor=0.62) # Warp mesh by deformation
plotter.add_mesh(warped) # we add the deformed shape to the plot

# we display the plot with axes and grid
plotter.show_axes()
plotter.view_xy()
plotter.show_bounds(grid='front', location='outer', all_edges=True)
plotter.show()

# 5) Export xdmf file and open it in Paraview

In [ ]:
# To have a name in Paraview
u_solution.name = "displacement_vector"

In [ ]:
# We export the mesh+solution to see it on Paraview
with dolfinx.io.XDMFFile(XXXX) as file:
    file.write_mesh(XXXX)
    file.write_function(XXXX)

# 6) Compute the L2-norm of the solution

$ L^2_\text{norm} = \sqrt{\int_\Omega u^2 dx}$

In [ ]:
# L2-norm of the solution
print('We compute the L2-norm of the solution')
print('For E_m = 0.8 , nu_m = 0.35 , E_i = 11.0 , nu_i = 0.3, R_i =1, R_e = 6.9, aspect_ratio = 1.0 , mesh_size = R_i/5, mesh_order = 1 ')
print('this norm should be equal to ~ 59')
np.sqrt( utils.assemble_scalar(XXX) )

In [ ]:
# On the diagonal x=y, u ~ -(x,x)*Emat/Einc in the inclusion
utils.evaluate_at_points(np.array([[0.2],[0.2],[0.0]]), u_solution)

In [ ]:
# On the diagonal x=y, u ~ -(x,x) in the matrix
utils.evaluate_at_points(np.array([[3.1],[3.1],[0.0]]), u_solution)

## 6.1) In the circular case, here is the analytical solution

In [ ]:
from eshelby import EshelbyDisk
solution = EshelbyDisk(V,R_e/R_i, E_i/E_m, nu_i, nu_m)
u_ref_func = solution.to_function(R_i)

In [ ]:
# analytical solution
utils.evaluate_at_points(np.array([[0.2],[0.2],[0.0]]), u_ref_func)

In [ ]:
# FEM solution
utils.evaluate_at_points(np.array([[0.2],[0.2],[0.0]]), u_solution)

In [ ]:
# Are they equal? Why?

# 7) Compute the strain tensor of the solution

In [ ]:
# We compute the strain tensor of the solution
eps_solution = eps(u_solution)

## Evaluating $ \epsilon_{ij} $

In [ ]:
V_eps = fem.functionSpace(domain,("XXX", 0))

In [ ]:
eps_xx_expr = dolfinx.fem.Expression(eps_solution[0,0], XXX)
eps_xx = dolfinx.fem.Function(XXX)
eps_xx.interpolate(XXX)

In [ ]:
# This should send back the strain at point (0.1,0.2)
utils.evaluate_at_points(np.array([[0.1],[0.2],[0.]]), eps_xx)

In [ ]:
# Plot using pyvista or Paraview

# Create pyvista grid
topology, cells, geometry = dolfinx.plot.vtk_mesh(u_solution.function_space)
function_grid = pyvista.UnstructuredGrid(topology, cells, geometry)

plotter = pyvista.Plotter() # create plotter

# Add eps_xx(x,y)
function_grid["eps_xx"] = eps_xx.x.array.real
plotter.add_mesh(function_grid, show_edges=True)

# we display the plot with axes and grid
plotter.show_axes()
plotter.view_xy()
plotter.show_bounds(grid='front', location='outer', all_edges=True)
plotter.show()

# 8) Do the same for $ \epsilon_{xy} $ and $ \epsilon_{yy} $ and 

In [ ]:
eps_xy_expr = XXX
eps_xy = XXX
eps_xy.interpolate(XXX)

In [ ]:
utils.evaluate_at_points(np.array([[0.5],[0.3],[0.]]), eps_xy)

In [ ]:
# In the limit of a very large matrix, in the case of a circular inclusion, 
# eps_xy(x,y) should be equal to
mu_m = E_m/(2*(1+nu_m))
mu_i = E_i/(2*(1+nu_i))
q = (3-4*nu_m)/(8*mu_m*(1-nu_m))
b = 1/(1+2*q*(mu_i-mu_m))
print('eps_xy_inclusion = ',-b)
# is it the case?

In [ ]:
# Plot using pyvista or Paraview

# Create pyvista grid
topology, cells, geometry = dolfinx.plot.vtk_mesh(u_solution.function_space)
function_grid = pyvista.UnstructuredGrid(topology, cells, geometry)

plotter = pyvista.Plotter() # create plotter

# add eps_xy(x,y)
function_grid["eps_xy"] = eps_xy.x.array.real
plotter.add_mesh(function_grid, show_edges=True)

# we display the plot with axes and grid
plotter.show_axes()
plotter.view_xy()
plotter.show_bounds(grid='front', location='outer', all_edges=True)
plotter.show()

In [ ]:
eps_yy_expr = XXX
eps_yy = XXX
eps_yy.interpolate(XXX)

In [ ]:
utils.evaluate_at_points(np.array([[2.5],[1.1],[0.0]]), eps_yy)

In [ ]:
# Plot using pyvista or Paraview

# Create pyvista grid
topology, cells, geometry = dolfinx.plot.vtk_mesh(u_solution.function_space)
function_grid = pyvista.UnstructuredGrid(topology, cells, geometry)

plotter = pyvista.Plotter() # create plotter

# add eps_yy(x,y)
function_grid["eps_yy"] = eps_yy.x.array.real
plotter.add_mesh(function_grid, show_edges=True)

# we display the plot with axes and grid
plotter.show_axes()
plotter.view_xy()
plotter.show_bounds(grid='front', location='outer', all_edges=True)
plotter.show()

# 9) Compute mean values over entire domain

\begin{equation}
<\varepsilon_{ij}> = \frac{\int_\Omega \varepsilon_{ij} \,\mathrm{d}x}{ \int_\Omega dx }
\end{equation}

In [ ]:
utils.assemble_XXX / area_domain

In [ ]:
utils.assemble_XXX / area_domain

In [ ]:
utils.assemble_XXX / area_domain

# 9.1) Compute mean values over inclusion

In [ ]:
mean_eps_xx = utils.assemble_XXX / area_inclusion

In [ ]:
mean_eps_xy = utils.assemble_XXX / area_inclusion

## How does this mean value change when the ratio $E_{incl}/E_{mat}$ is changing ?

In [ ]:
mean_eps_yy = utils.assemble_XXX / area_inclusion

# 9.2) Compute mean values over matrix

In [ ]:
utils.assemble_scalar(dolfinx.fem.form(eps_xx * dx(2))) / area_matrix

In [ ]:
utils.assemble_scalar(dolfinx.fem.form(eps_xy * dx(2))) / area_matrix

In [ ]:
utils.assemble_scalar(dolfinx.fem.form(eps_yy * dx(2))) / area_matrix

# 9.5) Compute the deviation from uniformity inside inclusion

\begin{equation}
deviation = \frac{\int_\Omega Abs(\varepsilon_{ij} - <\varepsilon_{ij}>) \,\mathrm{d}x }{ <\varepsilon_{ij}> }
\end{equation}

### eps_xx

In [ ]:
print(mean_eps_xx)

In [ ]:
# Explain / comment the result
deviation_xx = utils.assemble_XXX / mean_eps_xx

### eps_xy

In [ ]:
print(mean_eps_xy) 

In [ ]:
# Explain / comment the result
deviation_xy = utils.assemble_XXX / mean_eps_xy

### eps_yy

In [ ]:
print(mean_eps_yy) 

In [ ]:
# Explain / comment the result
deviation_yy = utils.assemble_XXX / mean_eps_yy

# 11) Plot u_y(x,0) using mathplotlib

# 12) Convergence study
How does the error decrease when the mesh size decreases?

If the mesh size is divided by 2, is the error also divided by 2?

Do the convergence exponents change when the inclusion is elliptical instead of circular?

# 13) Nondimensionalization
As usal for statics problems, we can freely choose a length unit, and a force unit.

Here we choose the length unit to be $R_i$, that is we set $R_i=1$.

We choose the force unit to be $E_m$, that is we set $E_m=1$.


###  13.1) We have set $R_e=6.9$ in our FEM computations. If, in the real world, the inclusion has radius $R_i=200$ microns, our FEM computations correspond to a matrix of which size? (6.9 microns, 6.9km, or 1.38 mm, or ... ?)

### 13.2) We have set $E_i = 11$ in our FEM computations. If, in the real world, the matrix has $E_m= 1.2$ GPa, our FEM computations correspond to an inclusion with which Young modulus? (11 GPa, or 13.2 GPa, ... ?)